# Import libraries

In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

# Basic libraries
import os
import pickle
import numpy    as np
import xgboost

# Data

In [2]:
# Load data
X = np.load('Data/data.npz', allow_pickle=True)['X']
y = np.load('Data/data.npz', allow_pickle=True)['y']

# Label encoding for targe class
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
pickle.dump(le, open('Model/Label_encoder.pkl', 'wb'))

## Prediction model

In [3]:
# Parameters
params = {
    'n_estimators'      : 500,
    'learning_rate'     : 1e-1,
    'max_depth'         : 10,
    'reg_alpha'         : 20, 
    'reg_lambda'        : 1.0, 
    'gamma'             : 1, 
    'min_child_weight'  : 2, 
    'max_leaves'        : 2, 
    'eval_metric'       : ['auc', 'aucpr', 'mlogloss']
}


# Setup model
model = xgboost.XGBClassifier(objective           = 'multi:softmax',  
                              n_jobs              = -1,                                   
                              validate_parameters = True, 
                              verbosity           = 1,
                              tree_method         = 'hist',
                              **params)

# Train model
weights = [y.shape[0] / np.where(y == i)[0].shape[0] for i in np.unique(y)]

model.fit(X, y,
          eval_set = [ (X, y) ],
          sample_weight = [weights[int(x)] for x in y],
          verbose = 50);

[0]	validation_0-auc:0.65337	validation_0-aucpr:0.17928	validation_0-mlogloss:2.02775
[50]	validation_0-auc:0.91622	validation_0-aucpr:0.67403	validation_0-mlogloss:1.05192
[100]	validation_0-auc:0.94565	validation_0-aucpr:0.71602	validation_0-mlogloss:0.77394
[150]	validation_0-auc:0.95853	validation_0-aucpr:0.73790	validation_0-mlogloss:0.63872
[200]	validation_0-auc:0.96664	validation_0-aucpr:0.75317	validation_0-mlogloss:0.55205
[250]	validation_0-auc:0.97190	validation_0-aucpr:0.76523	validation_0-mlogloss:0.49008
[300]	validation_0-auc:0.97536	validation_0-aucpr:0.77412	validation_0-mlogloss:0.44422
[350]	validation_0-auc:0.97794	validation_0-aucpr:0.78069	validation_0-mlogloss:0.40838
[400]	validation_0-auc:0.97985	validation_0-aucpr:0.78740	validation_0-mlogloss:0.37909
[450]	validation_0-auc:0.98148	validation_0-aucpr:0.79330	validation_0-mlogloss:0.35497
[499]	validation_0-auc:0.98272	validation_0-aucpr:0.79853	validation_0-mlogloss:0.33501


In [4]:
pickle.dump(model, open('Model/model.pkl', 'wb'))